In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 100

In [2]:
# ディレクトリ 内のメールファイルを読み込む

from fileRetrieval import path_list

directory_path = "enron/maildir/allen-p/inbox"
file_list, dir_list=path_list(directory_path)

In [3]:
# テキストファイルをデータフレームに格納する

from fileToDataFrame import fileToDataFrame

df = fileToDataFrame(file_list)

In [4]:
from toSentences import toSentences

sent_dict = {'Message-ID':[],
            'Sentence-ID':[],
            'Sentence':[]}

start = time.time()

for i,message in enumerate(df['Message']):
    sentences = toSentences(message)
    sent_dict['Message-ID'].extend([df['Message-ID'][i]] * len(sentences))
    sent_dict['Sentence-ID'].extend(range(len(sentences)))
    sent_dict['Sentence'].extend(sentences)

sentence_df = pd.DataFrame.from_dict(sent_dict,orient='columns')

end = time.time()
print(end-start,'s')

0.10239005088806152 s


In [5]:
# トリプルテーブルを作る

from tripleExtraction import tripleExtraction

triple_dict = {'Message-ID':[],
             'Sentence-ID':[],
             'Triple':[]}

start = time.time()

counter = 0
for i, sentence in enumerate(sentence_df['Sentence']):
    triples = tripleExtraction(sentence)
    triple_dict['Message-ID'].extend([sentence_df['Message-ID'][i]] * len(triples))
    triple_dict['Sentence-ID'].extend([sentence_df['Sentence-ID'][i]] * len(triples))
    triple_dict['Triple'].extend(triples)
    
triple_df = pd.DataFrame.from_dict(triple_dict,orient='columns')
        
end = time.time()
print(end-start,'s')

188.20253109931946 s


In [11]:
output = pd.merge(df[['Path','Message-ID','Subject']],pd.merge(sentence_df,triple_df,on=['Message-ID','Sentence-ID']),on='Message-ID')

In [13]:
output.to_csv('output/allen_p_inbox_triple.csv')